In [36]:
%pip install selenium
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests as rq
import re
import pandas as pd
import os
import pathlib


In [38]:
#cấu hình chrome driver
#config chrome driver options for crawl images
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-extensions")



In [39]:
#khởi tạo driver
driver = webdriver.Chrome(options=chrome_options)

In [40]:
# URL to the dermnet website
url = "https://dermnetnz.org/image-library"

# Navigating to the specified url in chrome
driver.get(url)
driver.implicitly_wait(5)

In [41]:
# Finding all the skin_disorders listed on the main page
skin_disorder_tag_main_page = driver.find_elements("xpath", '//a[@class="imageList__group__item"]')

# For loop to extract the skin disorder names and the link to the skin_disorders
disorder_names = []
link_list= []
for tag in skin_disorder_tag_main_page:
    disorder_names.append(tag.text)
    link_list.append(tag.get_attribute("href"))
disorder_names=list(map(str.lower,disorder_names))
# Previewing the lists created:
print(f'The first 10 skin condition names: \n {disorder_names[:10]}\n')
print(f'The first 10 links to skin condition image links:\n{link_list[:10]}')

The first 10 skin condition names: 
 ['acne affecting the back images', 'acne affecting the face images', 'acne and other follicular disorder images', 'acquired dermal macular hyperpigmentation images', 'acral lentiginous melanoma images', 'actinic keratosis affecting the face images', 'actinic keratosis affecting the hand images', 'actinic keratosis affecting the legs and feet images', 'actinic keratosis affecting the scalp images', 'actinic keratosis dermoscopy images']

The first 10 links to skin condition image links:
['https://dermnetnz.org/topics/acne-affecting-the-back-images', 'https://dermnetnz.org/topics/acne-face-images', 'https://dermnetnz.org/image-catalogue/acne-and-other-follicular-disorder-images', 'https://dermnetnz.org/topics/acquired-dermal-macular-hyperpigmentation-images', 'https://dermnetnz.org/images/acral-lentiginous-melanoma-images', 'https://dermnetnz.org/topics/actinic-keratosis-face-images', 'https://dermnetnz.org/topics/actinic-keratosis-affecting-the-hand-

In [42]:
# The number of skin_disorders listed in the website
print(f'The are {len(disorder_names)} skin conditions listed in the DermNet website.')

The are 294 skin conditions listed in the DermNet website.


In [43]:
# Creating a dataframe with two columns, the skin_disorder names and the links to the images of the skin disorders
name_link_df = pd.DataFrame({'skin_disorder_name': disorder_names, 'link': link_list})

# Saving the dataframe as a csv file
name_link_df.to_csv('Data/Link_Images.csv', index=False)

# Previewing the first five rows of the dataframe
name_link_df.head()

,skin_disorder_name,link
0,acne affecting the back images,https://dermnetnz.org/topics/acne-affecting-th...
1,acne affecting the face images,https://dermnetnz.org/topics/acne-face-images
2,acne and other follicular disorder images,https://dermnetnz.org/image-catalogue/acne-and...
3,acquired dermal macular hyperpigmentation images,https://dermnetnz.org/topics/acquired-dermal-m...
4,acral lentiginous melanoma images,https://dermnetnz.org/images/acral-lentiginous...


In [44]:
df=pd.read_csv('Data/Link_Images.csv')
acne_labels=list(df[df['skin_disorder_name'].str.contains('acne')]['skin_disorder_name'].unique())
acne_labels.remove('steroid acne images')
acne_labels.remove('acne and other follicular disorder images')
print(f'acne_labels: {acne_labels}')

# Labels representing eczema in Dermnet's scraped data.
eczema_labels = df[(df['skin_disorder_name'].str.contains('eczema')) | \
                         (df['skin_disorder_name'].str.contains('atopic dermatitis images')) |\
                         (df['skin_disorder_name'].str.contains('hand dermatitis images')) |\
                         (df['skin_disorder_name'] == 'dermatitis images') |\
                         (df['skin_disorder_name'].str.contains('nummular dermatitis images'))] \
                         ['skin_disorder_name'].unique()
print(f'zema_labels: {eczema_labels}')


acne_labels: ['acne affecting the back images', 'acne affecting the face images', 'facial acne images', 'infantile acne images']
zema_labels: ['atopic dermatitis images' 'atopic eczema images'
 'atopic flexural eczema images' 'dermatitis images'
 'discoid eczema images' 'eczema herpeticum images'
 'hand dermatitis images' 'nummular dermatitis images']


In [45]:
disease_df =[]
# Filtering the dataframe to only include acne labels
acne_df = df[df['skin_disorder_name'].isin(acne_labels)]
disease_df.append({'name':'acne','data':acne_df['link'].to_list()})
print(f'acne_df: {acne_df.shape}')
# Filtering the dataframe to only include eczema labels
eczema_df = df[df['skin_disorder_name'].isin(eczema_labels)]
disease_df.append({'name':'eczema','data':eczema_df['link'].to_list()})
print(f'eczema: {eczema_df.shape}')



acne_df: (4, 2)
eczema: (8, 2)


In [46]:
# create an empty list to store the url for each link/disease pair
image_disease = []
list_disease_name=[]
image_df = pd.DataFrame({'disease_name': list_disease_name, 'image_url': image_disease})
image_df.to_csv(f'Data/dermnetnz_Image.csv', index=False)
# loop through each link/disease pair
for disease in disease_df:
    # extract the disease name and the link

    name = disease['name']
    links = disease['data']
        
    image_disease = []
    list_disease_name=[]

    
    # create an empty list to store the image URLs
    driver = webdriver.Chrome(options=chrome_options)
    for link in links:
        try:
            # navigate to the link
            driver.get(link)
            print(f"extract {link}...")
            # wait for the page to load
            # time.sleep(5)
            # find all the elements on the page with the class "imageLinkBlock__item__image"
            skin_image_disorder = driver.find_elements("xpath", '//div[@class="imageLinkBlock__item__image"]/img')
            # loop through each element and find all the images within it
            for image in skin_image_disorder:
                # add each image URL to the image_disease list
                image_disease.append(image.get_attribute("src"))
                list_disease_name.append(name)
        except Exception as e:
            # if an error occurs, print the error message and move to the next link/disease pair
            print(f"Error occurred for {name}: {str(e)}")
            continue
        finally:
            pass
    # create a dataframe with the disease name and the image URL
    image_df = pd.DataFrame({'disease_name': list_disease_name, 'image_url': image_disease})
    # save the dataframe as a csv file
    image_df.to_csv(f'Data/dermnetnz_Image.csv', index=False,mode='a', header=False)
    driver.quit()

    



extract https://dermnetnz.org/topics/acne-affecting-the-back-images...
extract https://dermnetnz.org/topics/acne-face-images...
extract https://dermnetnz.org/topics/acne-face-images...
extract https://dermnetnz.org/topics/infantile-acne-images...
extract https://dermnetnz.org/images/atopic-dermatitis-images...
extract https://dermnetnz.org/images/atopic-dermatitis-images...
extract https://dermnetnz.org/topics/atopic-flexural-eczema-images...
extract https://dermnetnz.org/images/atopic-dermatitis-images...
extract https://dermnetnz.org/images/discoid-eczema-images...
extract https://dermnetnz.org/topics/eczema-herpeticum-images...
extract https://dermnetnz.org/topics/hand-dermatitis-images...
extract https://dermnetnz.org/images/discoid-eczema-images...


In [47]:
# create a dataframe from the image_disease list
dfs= pd.read_csv('Data/dermnetnz_Image.csv')
print(f'number of image: {len(dfs)}')
dfs.head()

# Turning all the disease_name to lower case and dropping duplicates
Result= dfs.drop_duplicates(keep='first')
print(f'number of image after drop duplicate: {len(Result)}')

# save the result dataframe to a CSV 
Result.to_csv('Data/dermnetnz_Image.csv', index=False)

number of image: 1184
number of image after drop duplicate: 621
